In [1]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from pyspark.sql.functions import *
from pyspark.sql.types import *
#sc.install_pypi_package("pandas==0.25.1") #Install pandas version 0.25.1 
#sc.install_pypi_package("matplotlib", "https://pypi.org/simple") #Install matplotlib from given PyPI repository

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1598696338484_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**DATA LOAD**


In [17]:
#load stores
customers  = sqlContext.read.json('s3://tfmbigdata/Files_Segunda_Entrega/stores.json')

#load customers
customers  = sqlContext.read.json('s3://tfmbigdata/Files_Segunda_Entrega/Customers.json')

#load products
products  = sqlContext.read.json('s3://tfmbigdata/Files_Segunda_Entrega/Products.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#load tickets

text = sc.textFile("s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part0.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part1.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part2.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part3.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part4.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part5.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part6.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part7.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part8.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part9.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part10.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part11.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part12.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part13.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part14.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part15.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part16.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part17.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part18.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part19.json")

tickets = spark.read.json(text)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


**RFM**

In [3]:
tickets.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cardtype: string (nullable = true)
 |-- customerid: string (nullable = true)
 |-- datekey: string (nullable = true)
 |-- extendedamount: double (nullable = true)
 |-- linenumber: long (nullable = true)
 |-- originalamount: double (nullable = true)
 |-- productid: long (nullable = true)
 |-- quantity: double (nullable = true)
 |-- storeid: string (nullable = true)
 |-- ticketid: string (nullable = true)
 |-- totaldiscount: double (nullable = true)

In [41]:
#transform data types
tickets = tickets.withColumn("quantity", tickets["quantity"].cast(IntegerType()))
tickets = tickets.withColumn("unitprice", tickets["extendedamount"].cast(DoubleType()))
tickets = tickets.withColumn("date", to_date("datekey", "yyyy-mm-dd"))

#calculate recency 
#max_df = tickets.select(max("date")).collect() 
max_df = tickets.select(max("date")).collect() 
max_val = max_df[0][0]
tickets = tickets.withColumn('max_date',lit(max_val))
tickets = tickets.withColumn("recencydays", expr("datediff(max_date, date)"))

#calculate amount spent for item 
tickets = tickets.withColumn("total", round(tickets["unitprice"] * tickets["quantity"], 2))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
#calculate recency frequency and money by customer

rfm_table = tickets.groupBy("customerid")\
                        .agg(min("recencydays").alias("recency"), \
                             countDistinct("ticketid").alias("frequency"), \
                             sum("total").alias("monetary"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:

rfm_table.coalesce(1).write.format('com.databricks.spark.csv').save('s3://aws-emr-resources-821081922153-us-east-2/notebooks/data/output_rfm.csv',header = 'true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
rfm_table.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-------+---------+------------------+
|       customerid|recency|frequency|          monetary|
+-----------------+-------+---------+------------------+
|77000000361655313|      0|       22| 754.4200000000001|
|77000000401454219|      0|       41|            977.94|
|77340000014507129|      1|       16|            213.64|
|77000000286756763|      1|       25| 4413.639999999999|
|77000000288269187|      0|       17|             532.2|
|77000000491611645|      4|        9|206.98999999999998|
|77000000426860362|      1|       33|1894.1399999999996|
|77000000065262986|      2|       23|            365.54|
|77340000015992587|      1|       26|            300.69|
|77000000087916366|      0|       15|174.70000000000002|
|77000000182437531|      3|       11|1423.9300000000003|
|77000000071401922|      1|        8|            678.07|
|77000000493458719|      0|       12|1700.7399999999998|
|77000000057973048|      1|       13|162.94000000000003|
|77000000271450260|      0|    

**RFM RECONCILIATION - TO BE DELETED**

In [10]:
test=tickets.groupBy("customerid").agg(countDistinct("ticketid").alias("count")).where('count > 1').sort('count', ascending=True).show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----+
|       customerid|count|
+-----------------+-----+
|77000000589581063|    8|
|77000000095667151|    8|
|77000000259375312|    8|
|77000000180113023|    8|
|77000000157129940|    8|
|77000000565181392|    8|
|77000000275807209|    8|
|77000000392952654|    8|
|77000000071401922|    8|
|77000000142728641|    8|
|77000000278692571|    8|
|77000000280875168|    8|
|77000000262693003|    8|
|77000000374570572|    8|
|77000000518911228|    8|
|77000000360465608|    8|
|77000000202695981|    8|
|77000000372388848|    8|
|77000000527675566|    8|
|77000000522589177|    8|
+-----------------+-----+
only showing top 20 rows

In [24]:
testing = tickets.select("customerid","ticketid","quantity","unitprice","date","max_date","recencydays"). \
filter((tickets.customerid  == "77000000361655313" )).orderBy("recencydays").show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+--------------------+--------+---------+----------+----------+-----------+
|       customerid|            ticketid|quantity|unitprice|      date|  max_date|recencydays|
+-----------------+--------------------+--------+---------+----------+----------+-----------+
|77000000361655313|10005047720190831...|       1|     0.47|2019-01-31|2019-01-31|          0|
|77000000361655313|10005047720190831...|       1|     0.99|2019-01-31|2019-01-31|          0|
|77000000361655313|10005047720190831...|       1|     1.59|2019-01-31|2019-01-31|          0|
|77000000361655313|10005047720190831...|       1|     0.99|2019-01-31|2019-01-31|          0|
|77000000361655313|10005047720190831...|       1|     2.19|2019-01-31|2019-01-31|          0|
|77000000361655313|10005047720190831...|       1|     0.99|2019-01-31|2019-01-31|          0|
|77000000361655313|10005047720190831...|       1|     0.85|2019-01-31|2019-01-31|          0|
|77000000361655313|10005047720190831...|       1|     1.39|2

In [25]:
rfm_table.filter((rfm_table.customerid  == "77000000361655313" )).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-------+---------+--------+----------+----------+----------+---------+
|       customerid|recency|frequency|monetary|r_quartile|f_quartile|m_quartile|rfm_score|
+-----------------+-------+---------+--------+----------+----------+----------+---------+
|77000000361655313|      0|       22|  754.42|         3|         3|         3|      333|
+-----------------+-------+---------+--------+----------+----------+----------+---------+

In [32]:
rfm_table.orderBy("recency").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-------+---------+------------------+----------+----------+----------+---------+
|       customerid|recency|frequency|          monetary|r_quartile|f_quartile|m_quartile|rfm_score|
+-----------------+-------+---------+------------------+----------+----------+----------+---------+
|77000000283740839|      0|       17|220.78000000000003|         3|         3|         1|      331|
|77000000288269187|      0|       17| 532.2000000000002|         3|         3|         3|      333|
|77000000512270026|      0|       27|            317.23|         3|         4|         2|      342|
|77000000475752145|      0|       31|1109.9099999999999|         3|         4|         4|      344|
|77000000087916366|      0|       15|             174.7|         3|         2|         1|      321|
|77000000109996185|      0|       14|            137.73|         3|         2|         1|      321|
|77000000289524270|      0|       12|450.98999999999995|         3|         2|         2|      322|


**RFM ANALYSIS - QUARTILES**

In [43]:
#create quartiles for each metric
r_quartile = rfm_table.approxQuantile("recency", [0.25, 0.5, 0.75], 0)
f_quartile = rfm_table.approxQuantile("frequency", [0.25, 0.5, 0.75], 0)
m_quartile = rfm_table.approxQuantile("monetary", [0.25, 0.5, 0.75], 0)
print(r_quartile)
print(f_quartile)
print(m_quartile)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.0, 1.0, 2.0]
[11.0, 16.0, 23.0]
[297.77000000000004, 506.15999999999997, 862.6800000000001]

In [44]:
#assing score from 1 to 4 dependending on quantile

rfm_table = rfm_table.withColumn("r_quartile", \
                                 when(col("recency") >= r_quartile[2] , 1).\
                                 when(col("recency") >= r_quartile[1] , 2).\
                                 when(col("recency") > r_quartile[0] , 3).\
                                 otherwise(4))

rfm_table = rfm_table.withColumn("f_quartile", \
                                 when(col("frequency") > f_quartile[2] , 4).\
                                 when(col("frequency") > f_quartile[1] , 3).\
                                 when(col("frequency") > f_quartile[0] , 2).\
                                 otherwise(1))

rfm_table = rfm_table.withColumn("m_quartile", \
                                 when(col("monetary") >= m_quartile[2] , 4).\
                                 when(col("monetary") >= m_quartile[1] , 3).\
                                 when(col("monetary") >= m_quartile[0] , 2).\
                                 otherwise(1))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
rfm_table = rfm_table.withColumn("rfm_score", concat(col("r_quartile"), col("f_quartile"), col("m_quartile")))
rfm_table.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-------+---------+------------------+----------+----------+----------+---------+
|       customerid|recency|frequency|          monetary|r_quartile|f_quartile|m_quartile|rfm_score|
+-----------------+-------+---------+------------------+----------+----------+----------+---------+
|77000000271450260|      0|       15|279.21999999999997|         4|         2|         1|      421|
|77000000198474444|      7|       10| 653.5300000000002|         1|         1|         3|      113|
|77000000288269187|      0|       17|             532.2|         4|         3|         3|      433|
|77000000152299922|      2|       21|            713.66|         1|         3|         3|      133|
|77000000087916366|      0|       15|             174.7|         4|         2|         1|      421|
|77000000485557584|      0|        9| 813.5699999999999|         4|         1|         3|      413|
|77000000191907447|      1|       31|            454.21|         2|         4|         2|      242|


In [46]:
rfm_table.coalesce(1).write.format('com.databricks.spark.csv').save('s3://aws-emr-resources-821081922153-us-east-2/notebooks/data/output_rfm.csv',header = 'true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…